In [4]:
# Attempt to generalize Conway's nimbers to characteristic p>2
'''
At each stage, we identify p^(p^k) with a root of the polynomial x^p - x - p^(p^(k-1))
This makes $\\mathbb{N}$ into a field isomorphic to the union of GF(p^(p^k)) over all k
'''

def deg(N : int, p : int = 2) -> int:
    '''
    Returns the largest D such that 
    p ** (p ** D) <= N
    under the p_nimber multiplication, the ordinal p^(p^D)
    is a field extension of Z/pZ of degree p^D
    '''
    D = 0
    if N < p:
        return 0
    while N // (p ** (p ** D)) != 0 :
        D += 1
    return D-1

def p_decomp(N : int, p : int = 2,  as_array = False, degree : int = -1) -> list:
    '''
    returns dict {'prime':p,'degree':degree,0:a_0,1:a_1,...,p-1:a_{p-1}} in the decomposition
    N = sum_{k=0}^{p-1} a_k * p^(k*(p^L)) , 
    where D=deg(N,p) and each a_k < p^(p^L).
    This is well-defined and unique by euclidean algorithm.
    '''
    if degree == -1:
        degree = deg(N,p)
    if p == 2:
        power = 1 << (1 << degree)
        decomp = {'prime':p,'degree':degree,0:N % power,1:N // power}
        if as_array:
            return [decomp[k] for k in range(p)] 
        return decomp
    else:
        power = p ** (p ** degree)
        decomp = {'prime':p,'degree':degree,0:N % power}
        for k in range(1,p):
            N = N // power
            decomp[k] = (N % power)
        if as_array:
            return [decomp[k] for k in range(p)]
        return decomp
    
def p_combine(decomp : dict) -> int:
    '''
    the inverse of p_decomp
    '''
    p = decomp['prime']
    degree = decomp['degree']
    if p == 2:
        return decomp[0] + (decomp[1] << (1 << degree))
    else:
        power = p ** (p ** degree)
        N = decomp[0]
        for k in range(1,p):
            N += decomp[k] * power
            power *= p ** (p ** degree)
        return N

In [ ]:
# sanity check
p = 3
N = 100
k = 1256
for n in range(N):
    decomp = p_decomp(k*n,p,as_array=False)
    decomp_array = p_decomp(k*n,p,as_array=True)
    print(f'{k*n} = {decomp}\n = {decomp_array}')
    print(f'{k*n} = {p_combine(decomp)}')

In [6]:
def int_log(N : int, base : int) -> int:
    '''
    returns floor(log_{base}(N)) 
    '''
    if N < base:
        return 0
    # for the highest power 'exp' s.t. base^exp <= N, find
    # the largest power of 2 that is less than or equal to exp
    total = 1 << deg(N, base)
    # now divide N to recursively find the binary expansion of exp
    N = N // (base ** total)
    while N != 0:
        total += 1 << deg(N, base)
        N = N // (base ** (1 << deg(N, base)))
    return total - 1

def nim_sum(N : int, M: int, p : int = 2) -> int:
    '''
    Returns the p-nimber sum of N and M
    '''
    # trivial cases
    if N == 0:
        return M
    elif M == 0:
        return N
    # if p = 2, then the sum is just the bitwise xor
    if p == 2:
        return N ^ M
    # otherwise, we do sum modulo p in each "p-bit" 
    if N < p and M < p:
        return (N + M) % p
    else:
        return ((N + M) % p) + p * nim_sum(N//p, M//p,p)

In [7]:
# special case of pi(D):=p**(p**D - 1) is easier to multiply
# because its decomp is [0,0,0,...,0,pi(D-1)]
# 
def pi(degree : int, p : int = 2) -> int:
    if degree == 0:
        return 1
    return p ** (p ** degree - 1)



In [46]:
# encode p_nimber multiplication recursively via matrices
import numpy as np

def pi_mult(N: int, D : int, p : int = 2) -> int:
    '''returns p**(p**D - 1) * N   w.r.t p-nimber multiplication
        whenever degree(N) < D
        otherwise the answer is incorrect, but we can recursively
        make sure the degree is less than D
    '''
    if D == 0:
        return N
    decomp_N = p_decomp(N,p,as_array=True,degree = D-1)
    decomp_product = {'prime':p,'degree':D-1}
    for k in range(p):
        decomp_product[k] = 0
    for k in range(p):
        for j in range(p):
            term1 = int((k == p-1) and (j == 0)) * pi_mult(decomp_N[j],D-1,p)
            term2 = int((k > 0) and (j == k)) * pi_mult(decomp_N[j],D-1,p)
            term3 = int(j == k+1) * pi_mult(pi_mult(decomp_N[j],D-1,p),D-1,p)
            sum = nim_sum(term3, nim_sum(term1,term2,p), p)
            decomp_product[k] =  nim_sum(decomp_product[k], sum, p)
    return p_combine(decomp_product)

def mult_matrix(N : int, p : int =2, D=-1) -> np.ndarray:
    '''returns the multiplication by N matrix'''
    if D == -1:
        D = deg(N,p)
    M = np.zeros((p,p),dtype=object)
    decomp_N = p_decomp(N,p,as_array=True,degree = D)
    for k in range(p):
        for j in range(p):
            condition1 = (k >= j)
            condition2 = (0 < k) and (k <= j)
            condition3 =  (j > k) and (0 <= p - (j - k)) and (p - (j - k) < p)

            if condition1:
                M[k,j] = nim_sum(M[k,j],decomp_N[k-j],p) 

            if condition2:
                M[k,j] = nim_sum(M[k,j],decomp_N[(p-1) - (j-k)],p)

            if condition3:
                M[k,j] = nim_sum(M[k,j], pi_mult(decomp_N[p - (j - k)],D,p), p)
    return M

def nim_product(N : int, M : int, p : int = 2) -> int:
    '''returns N * M w.r.t p-nimber multiplication'''
    # base case D = 0
    if N < p and M < p:
        return N * M % p
    # now inductively multiply assuming multiplication is already
    # defined for degrees less than D = max(deg(N,p),deg(M,p))
    D = max(deg(N,p),deg(M,p))
    bigger = max(N,M); smaller = min(N,M)

    # write min as a vector
    decomp_smaller = p_decomp(smaller,p,as_array=True,degree = D)
    # then write bigger as a multiplication matrix
    bigger_matrix = mult_matrix(bigger,p,D)

    # initialize the decomposition dictionary for the product
    decomp_product = {'prime':p,'degree':D}
    for k in range(p):
        decomp_product[k] = 0

    # do the matrix multiplication of mult_matrix(max) and decomp_min
    for i in range(p):
        for j in range(p):
            decomp_product[i] = nim_sum(decomp_product[i], nim_product(bigger_matrix[i,j], decomp_smaller[j], p), p)
    
    # finally, recombine the decomposition vector into a single integer
    return p_combine(decomp_product)
    

 
    

In [52]:
import sympy as sp
p=3
d = 1
N = p**(p**d)
M = np.zeros((N-1,N-1),dtype=object)
for i in range(1,N):
    for j in range(1,N):
        M[i-1,j-1] = nim_product(i,j,p)

sp.Matrix(M)

Matrix([
[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26],
[ 2,  1,  6,  8,  7,  3,  5,  4, 18, 20, 19, 24, 26, 25, 21, 23, 22,  9, 11, 10, 15, 17, 16, 12, 14, 13],
[ 3,  6,  9, 12, 15, 18, 21, 24,  4,  7,  1, 13, 16, 10, 22, 25, 19,  8,  2,  5, 17, 11, 14, 26, 20, 23],
[ 4,  8, 12, 16, 11, 24, 19, 23, 13, 17,  9, 25, 20, 21,  1,  5,  6, 26, 18, 22,  2,  3,  7, 14, 15, 10],
[ 5,  7, 15, 11, 13, 21, 26, 19, 22, 24, 20,  1,  3,  8, 16,  9, 14, 17, 10, 12, 23, 25, 18,  2,  4,  6],
[ 6,  3, 18, 24, 21,  9, 15, 12,  8,  5,  2, 26, 23, 20, 17, 14, 11,  4,  1,  7, 22, 19, 25, 13, 10, 16],
[ 7,  5, 21, 19, 26, 15, 13, 11, 17, 12, 10,  2,  6,  4, 23, 18, 25, 22, 20, 24, 16, 14,  9,  1,  8,  3],
[ 8,  4, 24, 23, 19, 12, 11, 16, 26, 22, 18, 14, 10, 15,  2,  7,  3, 13,  9, 17,  1,  6,  5, 25, 21, 20],
[ 9, 18,  4, 13, 22,  8, 17, 26, 12, 21,  3, 16, 25,  7, 11, 20,  2, 24,  6, 15, 19,  1, 10, 23,  5, 14],
[10, 20,  7, 17, 24,  5, 12, 22, 21, 

7625597484987